# ML: Exercício de Programação - Voting Classifiers

Neste exercício, vamos explorar o conceito de *Voting Classifiers* (Classificadores por Votação).

Usaremos o dataset **MNIST** (imagens de dígitos manuscritos 28×28, achatadas em 784 features), amplamente utilizado em classificação multiclasse (10 classes: dígitos de 0 a 9).

**Objetivo:**
1. Avaliar o desempenho de três classificadores individuais.
2. Criar e avaliar ensembles de *Hard Voting* e *Soft Voting*.
3. Comparar os resultados e avaliar o melhor modelo no conjunto de teste.

**Métrica de Avaliação:** Usaremos o **F1-Score (weighted)**, que é uma boa métrica para classificação multiclasse, pois equilibra precisão e recall.

Use `RANDOM_STATE = 42` em todo o código.

## 0. Configuração e Imports

Primeiro, vamos importar as bibliotecas necessárias.

In [1]:
import numpy as np
import pandas as pd

# Dataset
from sklearn.datasets import load_digits

# Pré-processamento e Métricas
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Ensemble
from sklearn.ensemble import VotingClassifier

# Configuração geral
RANDOM_STATE = 42

## 1. Preparação dos Dados

Carregamos o dataset MNIST e o separamos em conjuntos de treino e teste.

In [2]:
# Carregar o dataset MNIST
digits = load_digits()
X, y = digits.data, digits.target

# Dividir em conjuntos de treino e teste
TRAIN_SIZE = 500
X_train, X_test = X[:TRAIN_SIZE], X[TRAIN_SIZE:]
y_train, y_test = y[:TRAIN_SIZE], y[TRAIN_SIZE:]

# Escalar os pixels
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Remodelar as imagens para vetores 1D (28*28 = 784)
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

print(f"Formato dos dados de treino: {X_train.shape}")
print(f"Formato dos dados de teste: {X_test.shape}")

Formato dos dados de treino: (500, 64)
Formato dos dados de teste: (1297, 64)


## 2. Avaliação dos Modelos Individuais

Vamos definir nossos três classificadores diversos e avaliá-los usando validação cruzada (5-fold) no conjunto de treino.

In [3]:
# Instanciar os classificadores: Regressão Logística, Árvore de Decisão e SVM (parâmetros default).
log_clf = LogisticRegression()
tree_clf = DecisionTreeClassifier()
svm_clf =  SVC()

estimators = [('lr', log_clf), ('dt', tree_clf), ('svc', svm_clf)]

print("Resultados da Validação Cruzada (F1-Weighted) nos Modelos Individuais:")

for name, clf in estimators:
    # 1.2: Avalie cada classificador com validação cruzada de 5 folds e scoring='f1_weighted'
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring="f1_weighted")
    
    print(f"{name:>10}: Média F1-score (CV): {scores.mean():.4f}")

Resultados da Validação Cruzada (F1-Weighted) nos Modelos Individuais:
        lr: Média F1-score (CV): 0.9395
        dt: Média F1-score (CV): 0.7417
       svc: Média F1-score (CV): 0.9398


## 3. Avaliação do Voting Classifier (Hard Voting)

Agora, vamos combinar os três modelos usando *Hard Voting* e avaliar o ensemble.

In [4]:
# 1.3: Crie um VotingClassifier com os 3 estimadores e voting='hard'
hard_voting_clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', DecisionTreeClassifier(random_state=42)),
        ('svc', SVC(random_state=42))
    ],
    voting='hard'
)

# 1.4: Avalie o 'hard_voting_clf' com validação cruzada (5-fold, f1_weighted)
hard_scores = cross_val_score(hard_voting_clf, X_train, y_train, cv=5, scoring="f1_weighted")

print(f"Hard Voting: Média F1-score (CV): {hard_scores.mean():.4f}")

print("\n--- Votos para a primeira instância de treino ---")

# Treina o 'hard_voting_clf' no conjunto de treino COMPLETO (X_train, y_train)
hard_voting_clf.fit(X_train, y_train)

# Pega a primeira instância de treino para análise
X_first = X_train[:1]

# Mostra o voto de cada classificador individual para a X_first
votes = [(name, clf.predict(X_first)[0]) for name, clf in hard_voting_clf.named_estimators_.items()]

print(f"Votos individuais: {votes}")
print(f"Predição final (Hard): {hard_voting_clf.predict(X_first)[0]}")
print(f"Classe real: {y_train[0]}")

Hard Voting: Média F1-score (CV): 0.9500

--- Votos para a primeira instância de treino ---
Votos individuais: [('lr', 0), ('rf', 0), ('svc', 0)]
Predição final (Hard): 0
Classe real: 0


## 4. Avaliação do Voting Classifier (Soft Voting)

Vamos repetir o processo com *Soft Voting*. Lembre-se que para *Soft Voting*, todos os estimadores precisam ter o método `predict_proba()`.

**Importante:** O `SVC` por padrão não tem `predict_proba()` habilitado. Precisamos recriar nosso estimador `SVC` com `probability=True`.

In [6]:
# Recriar estimadores para soft voting
log_clf_soft = LogisticRegression(random_state=RANDOM_STATE)
tree_clf_soft = DecisionTreeClassifier(random_state=RANDOM_STATE)

# Criar um novo SVC com probability=True e random_state=RANDOM_STATE
svm_clf_soft = SVC(random_state=RANDOM_STATE, probability=True)

estimators_soft = [('lr', log_clf_soft), ('dt', tree_clf_soft), ('svc', svm_clf_soft)]

# 1.8: Crie um VotingClassifier com os novos estimadores e voting='soft'
soft_voting_clf = VotingClassifier(estimators=estimators_soft, voting='soft')

# 1.9: Avalie o 'soft_voting_clf' com validação cruzada (5-fold, f1_weighted)
soft_scores = cross_val_score(soft_voting_clf, X_train, y_train, cv=5, scoring="f1_weighted")

print(f"Soft Voting: Média F1-score (CV): {soft_scores.mean():.4f}")

print("\n--- Probabilidades para a primeira instância de treino ---")

# Treina o 'soft_voting_clf' no conjunto de treino COMPLETO (X_train, y_train)
soft_voting_clf.fit(X_train, y_train)

# Pega a primeira instância de treino para análise
X_first = X_train[:1]

# Mostra as probabilidades (predict_proba) de cada classificador para X_first
print("Probabilidades individuais:")
for name, clf in soft_voting_clf.named_estimators_.items():
    print(f"{name:>5}: {clf.predict_proba(X_first)[0].round(3)}")

# Mostra as probabilidades médias (predict_proba) do 'soft_voting_clf' para X_first
print(f"\nProbs médias (Soft): {soft_voting_clf.predict_proba(X_first)[0].round(3)}")
print(f"Predição final (Soft): {soft_voting_clf.predict(X_first)[0]}")
print(f"Classe real: {y_train[0]}")

Soft Voting: Média F1-score (CV): 0.9218

--- Probabilidades para a primeira instância de treino ---
Probabilidades individuais:
   lr: [0.994 0.    0.    0.    0.    0.    0.    0.001 0.    0.004]
   dt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  svc: [0.939 0.002 0.003 0.007 0.005 0.009 0.005 0.007 0.006 0.017]

Probs médias (Soft): [0.977 0.001 0.001 0.002 0.002 0.003 0.002 0.003 0.002 0.007]
Predição final (Soft): 0
Classe real: 0


## 5. Avaliação Final no Conjunto de Teste

Compare os resultados de F1-score da validação cruzada (Modelos Individuais, Hard Voting e Soft Voting) e identifique o melhor modelo. Em seguida, avalie **apenas o melhor modelo** no conjunto de teste (`X_test`, `y_test`) para obter a estimativa final de desempenho.

In [9]:
# 1.13: Com base nos scores de CV anteriores, qual foi o melhor modelo?
# Use o melhor modelo (já treinado no conjunto de treino completo) para fazer predições no X_test

best_model = hard_voting_clf
y_pred_test = best_model.predict(X_test)

# 1.14: Calcule o f1_score (weighted) final no conjunto de teste
f1_final = f1_score(y_test, y_pred_test, average='weighted')

print(f"F1-score final do melhor modelo/ensemble ('{best_model.__class__.__name__}') no conjunto de teste: {f1_final:.4f}")

F1-score final do melhor modelo/ensemble ('VotingClassifier') no conjunto de teste: 0.9145
